# County NAICS Files
After running the notebook, delete the python/data_raw folder<br>
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html<br>
This notebook is a filtered and edited version of us_econ.ipynb for generating yearly data.<br>
For earlier code that generates averaged data aver years please refer us_econ.ipynb.<br>
This notebook will start generating data from year 2017.



In [1]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
import datetime
endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [2]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
#print(repo_dir)

raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)

In [3]:
# Load the state FIPS codes key
state_fips = pd.read_csv('../../../us/id_lists/state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [4]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#

def get_county_cbp(fips, state, years):
    count = 0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year >= 2000 and year <= 2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2003 and year <=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2008 and year <= 2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2012 and year <= 2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [5]:
#Years Initialization for data generation
startyear = 2017
endyear = 2021

In [6]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     years=range(startyear,endyear)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2017
  > Finished CSV for year2017
Getting data for state: Alabama	year: 2018
  > Finished CSV for year2018
Getting data for state: Alabama	year: 2019
  > Finished CSV for year2019
Getting data for state: Alabama	year: 2020
  > Finished CSV for year2020
Getting data for state: Alaska	year: 2017
  > Finished CSV for year2017
Getting data for state: Alaska	year: 2018
  > Finished CSV for year2018
Getting data for state: Alaska	year: 2019
  > Finished CSV for year2019
Getting data for state: Alaska	year: 2020
  > Finished CSV for year2020
Getting data for state: Arizona	year: 2017
  > Finished CSV for year2017
Getting data for state: Arizona	year: 2018
  > Finished CSV for year2018
Getting data for state: Arizona	year: 2019
  > Finished CSV for year2019
Getting data for state: Arizona	year: 2020
  > Finished CSV for year2020
Getting data for state: Arkansas	year: 2017
  > Finished CSV for year2017
Getting data for state: Arkansas	year: 2018
  > Finish

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS files that we downloaded

In [7]:
# Load the data from startyear

def load_all_states(bea_data_dir):
    
    for i in range(startyear,endyear):

        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:

            # variable selection based on census year
            naics_str = "NAICS2012" if i < 2017 else "NAICS2017"
            naics_ttl = "NAICS2012_TTL" if i < 2017 else "NAICS2017_LABEL"
            geo_ttl = "GEO_TTL" if i < 2017 else "NAME"

            df = pd.read_csv(f,encoding='latin-1',dtype={naics_str: str})
            if 'Unnamed: 11' in df.columns:
                df=df.drop("Unnamed: 11", axis=1)
            if 'Unnamed: 10' in df.columns:
                df=df.drop("Unnamed: 10", axis=1)

            # renaming columns so similar data from 2012 census & 2017 census are entered into appropriate columns
            df = df.rename(columns={"fips": "id", naics_str: "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab", naics_ttl:"NAICS_TTL", geo_ttl:"GEO_TTL"})
            naics_str = "relevant_naics"
            naics_ttl = "NAICS_TTL"
            geo_ttl = "GEO_TTL"

            df['is5'] = df[naics_str].apply(lambda x: 'True' if len(x) == 5 else 'False')

            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df[naics_str]
            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])

            yield df
    
df = pd.concat(load_all_states(county_data_dir)).drop("is5", axis=1)

#df

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,county,NAICS_Sector
0,0500000US51127,"New Kent County, Virginia",127,2017,237310,"Highway, street, and bridge construction",4,47,4387,51,127,23
1,0500000US51127,"New Kent County, Virginia",127,2017,238,Specialty trade contractors,58,295,15102,51,127,23
2,0500000US51127,"New Kent County, Virginia",127,2017,2381,"Foundation, structure, and building exterior c...",12,79,6415,51,127,23
3,0500000US51127,"New Kent County, Virginia",127,2017,23814,Masonry contractors,6,24,1010,51,127,23
4,0500000US51127,"New Kent County, Virginia",127,2017,238140,Masonry contractors,6,24,1010,51,127,23
...,...,...,...,...,...,...,...,...,...,...,...,...
2876,0500000US10001,"Kent County, Delaware",1,2020,81392,Professional organizations,4,12,273,10,1,81
2877,0500000US10001,"Kent County, Delaware",1,2020,813920,Professional organizations,4,12,273,10,1,81
2878,0500000US10001,"Kent County, Delaware",1,2020,81393,Labor unions and similar labor organizations,16,42,3157,10,1,81
2879,0500000US10001,"Kent County, Delaware",1,2020,813930,Labor unions and similar labor organizations,16,42,3157,10,1,81


In [8]:
df=df.drop("county", axis=1)

### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [9]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

In [10]:
# NOTE If this block is run please delete the generated file before pushing into repo (file size too large)
#df.to_csv("allll.csv")